In [1]:
from monomial_neural_network import *
from experiment import *

In [ ]:
d = 4 # input data dimension
teacher_k = [10] # teacher model hidden layer sizes - 3 layers with increasing number of neurons
# teacher_k = [10] # teacher model hidden layer sizes - 1 layer with 10 neurons

teacher_model = generate_teacher_model_noOutWeight(d, teacher_k)
print(teacher_model)

n = 11 # number of data points
# same data dimension d as before

data = generate_data(n, d, teacher_model)

# verify that the data is generated correctly
print(data)
print(data[0][0], data[1][0], teacher_model.evaluate(data[0][0]))

student_k = [10] # student model hidden layer sizes - 2 layers with increasing number of neurons
student_model = generate_student_model_noOutWeight(d, student_k)
print(student_model)



MonomialNeuralNetwork_noOutputWeight(
  (layers): Sequential(
    (0): Linear(in_features=4, out_features=10, bias=False)
    (1): Monomial()
    (2): Linear(in_features=10, out_features=1, bias=False)
  )
)
(tensor([[ 0.1857,  1.9637,  2.1015, -0.7523],
        [ 1.4161,  1.1846, -0.5858,  0.1405],
        [ 0.9466,  0.0789,  0.2166,  0.4672],
        [ 0.7652,  1.1856, -0.4130, -1.6143],
        [-1.8120,  1.7171, -1.1413,  1.4184],
        [-1.1497, -2.0946, -0.6056,  2.8334],
        [ 0.4669,  0.6362, -1.2938,  0.9946],
        [-0.3734, -0.5143, -0.8752, -2.6766],
        [-1.5875,  1.1046, -0.4019,  2.1080],
        [ 1.7395,  0.2512,  0.5841,  0.7440]]), tensor([[10.1249],
        [ 7.9558],
        [ 2.5299],
        [ 7.6739],
        [36.3577],
        [13.0504],
        [ 6.0467],
        [14.9172],
        [29.4708],
        [ 8.6245]]))
tensor([ 0.1857,  1.9637,  2.1015, -0.7523]) tensor([10.1249]) tensor([10.1249])
MonomialNeuralNetwork_noOutputWeight(
  (layers): Sequen

In [31]:
# train the student
student_model, losses = train(
    model = student_model, 
    x_train = data[0], 
    y_train= data[1], 
    num_epochs = 40000, 
    lr = 1e-3
    )

starting training
Epoch [0/40000], Loss: 110.37209
Epoch [100/40000], Loss: 85.65590
Epoch [200/40000], Loss: 65.41264
Epoch [300/40000], Loss: 48.85165
Epoch [400/40000], Loss: 35.36958
Epoch [500/40000], Loss: 24.55598
Epoch [600/40000], Loss: 16.15194
Epoch [700/40000], Loss: 9.96635
Epoch [800/40000], Loss: 5.76595
Epoch [900/40000], Loss: 3.17901
Epoch [1000/40000], Loss: 1.70527
Epoch [1100/40000], Loss: 0.88796
Epoch [1200/40000], Loss: 0.45385
Epoch [1300/40000], Loss: 0.23012
Epoch [1400/40000], Loss: 0.10620
Epoch [1500/40000], Loss: 0.05237
Epoch [1600/40000], Loss: 0.03095
Epoch [1700/40000], Loss: 0.02072
Epoch [1800/40000], Loss: 0.01489
Epoch [1900/40000], Loss: 0.01111
Epoch [2000/40000], Loss: 0.00846
Epoch [2100/40000], Loss: 0.00653
Epoch [2200/40000], Loss: 0.00508
Epoch [2300/40000], Loss: 0.00400
Epoch [2400/40000], Loss: 0.00318
Epoch [2500/40000], Loss: 0.00257
Epoch [2600/40000], Loss: 0.00210
Epoch [2700/40000], Loss: 0.00174
Epoch [2800/40000], Loss: 0.00147


In [35]:
# lets look at the weight matrices
# print(student_model.layers[0].weight)
student_w = student_model.layers[0].weight.detach().numpy()
# student_v = student_model.layers[2].weight.detach().numpy()
# print(student_w)
teacher_w = teacher_model.layers[0].weight.detach().numpy()
# teacher_v = teacher_model.layers[2].weight.detach().numpy()
# print(student_w)
# print(teacher_w)
# print(student_v)
# print(teacher_v)
# print(np.squeeze(student_v))
# A = student_w.T @ np.diag(np.squeeze(student_v)) @ student_w
# Astar = teacher_w.T @ np.diag(np.squeeze(teacher_v)) @ teacher_w
A = student_w.T @ student_w
Astar = teacher_w.T @ teacher_w
print(A)
print(Astar)
eigA, eigvecA = np.linalg.eig(A)
print(eigA)
# print(Astar)
eigAstar, eigvecAstar = np.linalg.eig(Astar)
print(eigAstar)
e = pop_loss(student_model, teacher_model, d=d, N=10000)
# print(y_teach)
# print(y_stud)
# print(torch.norm(y_teach))
print("Population loss = "+str(e))

[[ 3.3600945  -1.1257662  -0.39243376 -0.816296  ]
 [-1.1257662   2.1519432  -0.49748465  0.74801093]
 [-0.39243376 -0.49748465  2.2769542   0.6069933 ]
 [-0.816296    0.74801093  0.6069933   1.3959402 ]]
[[ 3.3376353  -1.0867145  -0.35086843 -0.8388499 ]
 [-1.0867145   2.1277146  -0.5053588   0.76081246]
 [-0.35086843 -0.5053588   2.3117304   0.61279106]
 [-0.8388499   0.76081246  0.61279106  1.3862303 ]]
[4.4600277 1.4670224 0.5730873 2.6847944]
[4.418686  1.4791214 0.5549545 2.7105486]
Population loss = 0.021259774


In [5]:
test_vec = np.array([[1,2]]).T
print(test_vec.T@A@test_vec)
# print(test_vec.T@Astar@test_vec)
test_vec_torch = torch.from_numpy(test_vec.T).type(torch.FloatTensor)
print(test_vec_torch)
print(student_model(test_vec_torch))
print(student_model.layers[2].weight)
print(teacher_model.layers[2].weight)
# print(teacher_model(test_vec_torch))

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 4 is different from 2)